In [41]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [42]:
import hydra
from hydra.utils import instantiate
from hydra import compose, initialize
from omegaconf import OmegaConf
import torchaudio, torch
import matplotlib.pyplot as plt

In [38]:
torch.hann_window(1024)

tensor([0.0000e+00, 9.4175e-06, 3.7640e-05,  ..., 8.4698e-05, 3.7640e-05,
        9.4175e-06])

In [39]:
with initialize("../configs/"):
    cfg = compose(config_name="train")
    print(OmegaConf.to_yaml(cfg))


original_work_dir: ${hydra:runtime.cwd}
data_dir: ${original_work_dir}/data/
print_config: true
ignore_warnings: true
train: true
test: false
seed: null
name: default
audio:
  transform:
    _target_: torchaudio.transforms.MelSpectrogram
    sample_rate: 32000
    n_fft: 1024
    win_length: 1024
    hop_length: 512
    f_min: 0.0
    f_max: 16000.0
    n_mels: 128
    window_fn:
      _target_: hydra.utils.get_method
      path: torch.hann_window
datamodule:
  _target_: src.datamodules.mnist_datamodule.MNISTDataModule
  data_dir: ${data_dir}
  batch_size: 64
  train_val_test_split:
  - 55000
  - 5000
  - 10000
  num_workers: 0
  pin_memory: false
model:
  _target_: src.models.mnist_module.MNISTLitModule
  lr: 0.001
  weight_decay: 0.0005
  net:
    _target_: src.models.components.simple_dense_net.SimpleDenseNet
    input_size: 784
    lin1_size: 256
    lin2_size: 256
    lin3_size: 256
    output_size: 10
callbacks:
  model_checkpoint:
    _target_: pytorch_lightning.callbacks.ModelC

In [40]:
a = instantiate(cfg.audio.transform)
a

MelSpectrogram(
  (spectrogram): Spectrogram()
  (mel_scale): MelScale()
)

In [31]:
hydra.utils.get_method("torch.hann_window")

<function _VariableFunctionsClass.hann_window>